# Codec Evaluator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import librosa
import numpy as np
import sys
import pickle
import time
import IPython.display as ipd

---
## Import Codec

In [ ]:
# path to your codec.py here
codec_path = 
sys.path.insert(1, codec_path)
from codec import encode, decode

---
## Import Audio File
*Load in an audio file to test with codec.*

In [ ]:
# select a clip number 1–7 to use the provided test clips, or provide your own audio path
clip_num =
audio_path = "/content/drive/MyDrive/ECES-434 Sessions/Codec Project/audio/clip_" + str(clip_num) + ".wav"

In [ ]:
def load_cd_quality_audio(filename):
    audio, sr = librosa.load(filename, sr = 44100, dtype='float_')
    max_int_value = 2**15 - 1
    audio *= max_int_value
    audio = audio.astype('int16')
    return audio
    
x = load_cd_quality_audio(audio_path) 

---
## Runtime
*Encode and decode the audio. Time the processes.*

In [ ]:
encodeStartTime = time.time()
x_encoded = encode(x)
ert = time.time() - encodeStartTime

decodeStartTime = time.time()
x_decoded = decode(x_encoded)
drt = time.time() - decodeStartTime

---
## Format Check
*Check to make sure the decoded audio is mono 16 bit.*

In [ ]:
def check_decoded_output(d):
    if type(d) != np.ndarray:
        print('ERROR: Your decoded signal is not a numpy array!')
    elif d.dtype != 'int16':
        print('ERROR: Your decoded signal does not contain 16 bit integers!')
    elif len(d.shape) != 1:
        print('ERROR: Your signal is not a 1-dimensional vector!')
    else:
        print('Your decoded signal passes the format check.')
        
check_decoded_output(x_decoded)

---
## Compression Ratio
*Compare the sizes of the original and encoded structures.*

In [ ]:
def compressionRatio(original, encoded):
    o_str = pickle.dumps(original)
    e_str = pickle.dumps(encoded)
    return sys.getsizeof(o_str)/sys.getsizeof(e_str)

cr = compressionRatio(x, x_encoded)

---
## SNR
*Compare the original signal content to the decoded version*

In [ ]:
def signalToNoise(original, decoded):
    
    original = original.astype('float_')
    decoded = decoded.astype('float_')
    
    # force the signals to be the same length
    diff = len(original) - len(decoded)
    if diff < 0:
        decoded = decoded[:diff]
    elif diff > 0:
        decoded = np.append(decoded, np.zeros( (diff,1) ) )
        
    # compute snr
    signal = sum(np.power(original,2))
    noise = sum(np.power(original - decoded,2))
    
    return np.mean(10 * np.log10(signal/noise))

snr = signalToNoise(x, x_decoded)

---
## Evaluate Codec
*Print out evalutation of codec. Listen to the results*

In [ ]:
print("Compression Ratio: ", str(round(cr,4)))
print()
print("Total Runtime: ", str(round(ert + drt,4)))
print("\tEncode Runtime: ", str(round(ert,4)))
print("\tDecode Runtime: ", str(round(drt,4)))
print()
print("SNR: ", str(round(snr,4)))


In [ ]:
# Orignal
ipd.Audio(x, rate = 44100)

In [ ]:
# Decoded
ipd.Audio(x_decoded, rate = 44100)